Dies ist der erste Test, um den Pseudo Random Number Generator ChaCha20 mit einem Machine Learning Algorithmus zu knacken.

In [ ]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00


Ich importiere hier einerseits tensorflow(1) für das Machine Learning Model, numpy(2) für die Berechnungen oder ähnliches, secrets(3) für die Nachricht mit sicher verteilten zufälligen bits und von Crypto.Cipher ChaCha20 und den Keysteam(4), für die Dokumentation siehe :
1. https://www.tensorflow.org/guide
2. https://numpy.org/
3. https://docs.python.org/3/library/secrets.html#module-secrets
4. https://pycryptodome.readthedocs.io/en/latest/src/cipher/chacha20.html

In [ ]:
#import
import numpy as np
import csv
import pandas as pd
from transformers import pipeline
from Crypto.Cipher import ChaCha20 as Cha
from Crypto.Random import get_random_bytes
from transformers import T5Tokenizer, T5Model, T5Config
from datasets import load_dataset

This Section will genrate the data Using the PyCryptodome Library with a train set of 100000 examples and a test set containing 100000

In [ ]:
# Generate 10,000 pairs of input-output data
train_size = 90000
test_size = 10000
val_size = 10000

full_size = train_size + test_size + val_size
plaintext = b''


number_of_keys = 1
keys = []
ciphers = []
known_blocks = []
prediction_blocks = []
nonces = []
data = []

blocks = 3
#each Block has 512 Bit or 64 Bytes, so for each block we want we need to make sure we encrypt 64 Byte only zeros
for i in range(0, blocks *64) :
  plaintext = plaintext + b'\x00'

"""generate the stream :
    In each Loop we generate a new Cipher Object, and out of it a ChaCha Object
    then we go to the position 0 in the Stream (try it with and without this)
    and encrypt the 1536 bit zeros
    a new key is generated in such a way that the texts encrypted with each key are evenly distributed
"""

counter = 0
for i in range(0, test_size + train_size + val_size) :
    if(i % (full_size / number_of_keys) == 0) :
         keys.append(get_random_bytes(32))
         counter = counter + 1

    ciphers.append(Cha.new(key=keys[counter-1]))
    ciphers[i].seek(0)
    ciphetext = ciphers[i].encrypt(plaintext)
    nonce = ciphers[i].nonce
    ciphetext_bits = ''.join(format(byte,'08b') for byte in ciphetext)
    nonce_bits = ''.join(format(byte, '08b') for byte in nonce)
    nonces.append(nonce_bits)
    known_blocks.append(ciphetext_bits[:1024])
    prediction_blocks.append(ciphetext_bits[1024:])
    data.append([ ciphetext_bits[:1024], ciphetext_bits[1024:],  nonce_bits])

train_set = {'generated_block' : known_blocks[:train_size] ,'next_Block' : prediction_blocks[:train_size] ,'nonce' : nonces[:train_size]}
test_set  = {'generated_block' : known_blocks[train_size:(train_size+test_size)] ,'next_Block' : prediction_blocks[train_size:(train_size+test_size)] ,'nonce' : nonces[train_size:(train_size+test_size)]}
validation_set = {'generated_block' : known_blocks[(train_size+test_size):] ,'next_Block' : prediction_blocks[(train_size+test_size):] ,'nonce' : nonces[(train_size+test_size):]}

df = pd.DataFrame.from_records(data)


Dieser Block ist für das erstellen des CSV

In [ ]:
header = ['generated_block','next_Block','nonce']
with open(r'data/data_dump.csv', 'w', encoding='UTF8',newline='') as fp:
    writer = csv.writer(fp)
    writer.writerow(header)
    for text in data :
        writer.writerow(text)


In [ ]:
!mkdir data

This next part is for the creation of the dataset and training of the tokenizer

In [ ]:
dataset = load_dataset("csv", data_files = "data/data_dump.csv")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
adjusted_dataset = dataset['train'].train_test_split(train_size= 10/11, seed =1729)
full_dataset = adjusted_dataset["train"].train_test_split(train_size = 0.9, seed = 42)
full_dataset["validation"] = adjusted_dataset.pop("test")
print(full_dataset["test"].shuffle().select(range(10)))
dataset['train'][5]

Dataset({
    features: ['generated_block', 'next_Block', 'nonce'],
    num_rows: 10
})


{'generated_block': '0111001011111101110001111111110000011101111011110110111001000000010011111111001100111011111010100000110101111011100000011100010000111100111010111010011110101001000100100010001000111100100101100100101111110101011010001101010000111010110100011111010010000101110100101001101110011001010110110001111000001001100001001110001001011011001110110111011111101101001100110001100100101100110101001001000101011100110010100111100100010111011110011100001011010111100101101001101000000000001111111000000101011110111001001011100011010110110010110000001100101101111110011101011110111110101101011111001100001111001011101011000010101000111010101011111111100100111111110111011101100110010010111111100100001101011000110011000011111101011100110010000010011010001000001110001000011001100010101110001101100011111011100101100000101111000001100010010111011110110011100000100111100101101100110111110011101011000001110110101111111101000001110101001011101111111010011110011111011101101011110110100000000010011

In [ ]:
def get_training_corpus():
  dataset = full_dataset["train"]
  for start in range(0,len(dataset),1000) :
    samples = dataset[start : start +1000]
    yield samples['generated_block']

In [ ]:
print(model_inputs)
print(data[0])

print(len(data[0][0])-len(tester[0][0]))

for i in range(0, len(data)) :
  if(len(data[i][0])-len(tester[i][0]) != 512):
    print("fehler")

{'input_ids': [[7088, 632, 2915, 15866, 19277, 2915, 15866, 19277, 1714, 15866, 1714, 2915, 2313, 15866, 2915, 2915, 19621, 19277, 19621, 19568, 1714, 1714, 1714, 15866, 19568, 1714, 19277, 2596, 15866, 2915, 19277, 1714, 15866, 2915, 17465, 2915, 17465, 16824, 15866, 1714, 2915, 16824, 19277, 2596, 2915, 2915, 19621, 19277, 15866, 19277, 2915, 2596, 15866, 1714, 19277, 2596, 2915, 19277, 19277, 19277, 16824, 2313, 19568, 16824, 19277, 1714, 1714, 16824, 536, 2915, 19277, 19277, 1714, 19621, 17465, 15866, 17465, 15866, 2915, 1714, 2915, 19277, 1714, 19277, 2915, 16824, 2596, 1714, 15866, 19277, 2915, 15866, 1714, 1714, 1714, 16824, 19277, 19277, 17465, 16824, 16824, 15866, 19277, 2596, 15866, 2313, 2915, 2915, 2313, 19277, 19277, 2915, 2596, 19621, 16824, 15866, 1714, 19621, 19277, 19621, 15866, 2915, 1206, 16824, 1714, 2915, 2915, 1714, 16824, 19277, 15866, 19277, 15866, 1714, 19621, 15866, 2915, 1206, 1714, 2596, 19621, 15866, 15866, 19277, 2915, 17465, 19621, 16824, 16824, 19621, 29

NameError: ignored

This next part will use the T5Model and Tokenizer

In [ ]:
from transformers import AutoTokenizer

checkpoint = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
print(T5Config())

T5Config {
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.33.2",
  "use_cache": true,
  "vocab_size": 32128
}



In [ ]:
new_tokenizer = tokenizer.train_new_from_iterator(get_training_corpus(),52000)

In [ ]:
config = T5Config(num_layers =2)
print(config)

T5Config {
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 2,
  "num_heads": 8,
  "num_layers": 2,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.33.2",
  "use_cache": true,
  "vocab_size": 32128
}



In [ ]:
model_default = T5Model.from_pretrained(checkpoint)
my_model = T5Model.from_pretrained(checkpoint,config = config, ignore_mismatched_sizes=True)

Some weights of T5Model were not initialized from the model checkpoint at google/flan-t5-small and are newly initialized: ['encoder.block.0.layer.1.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'encoder.block.1.layer.1.DenseReluDense.wi.weight', 'decoder.block.1.layer.2.DenseReluDense.wi.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of T5Model were not initialized from the model checkpoint at google/flan-t5-small and are newly initialized because the shapes did not match:
- encoder.block.0.layer.0.SelfAttention.q.weight: found shape torch.Size([384, 512]) in the checkpoint and torch.Size([512, 512]) in the model instantiated
- encoder.block.0.layer.0.SelfAttention.k.weight: found shape torch.Size([384, 512]) in the checkpoint and torch.Size([512, 512]) in the model instantiated
- encoder.block.0.layer.0.SelfAttention.v.weight: found shape torch.Size([384, 512]) in the

In [ ]:
prefix = "predict the sequence with this input"
max_input_length = 1024
max_target_length = 512

checkpoint = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_input(example) :
    generated_blocks = example['generated_block']
    print(type(generated_blocks[1]))
    predicted_blocks = example['next_Block']
    nonce = example['nonce']

    inputs =[]

    for i in range(0,len(generated_blocks)) :
      inputs.append([prefix , generated_blocks[i] + " Nonce: " + nonce[i]])

    print(inputs[1])
    model_inputs = tokenizer(inputs,truncation=True)
    model_inputs = tokenizer(predicted_blocks, truncation = True)

    model_inputs['outputs'] = model_outputs
    return model_inputs

In [ ]:

def tokenize_example(example) :
  return tokenizer(prefix + example['genrated_block'] + " Nonce " + example['nonce'], example['next_Block'], truncation=True)

In [ ]:
#tokenizer.decode(tokenizer.convert_tokens_to_ids(tokenized_data[1].tokens))
new_data = raw_data.map(tokenize_example, batched=True)

AttributeError: ignored

In [ ]:
#Testing
for n in data :
  if len(n[0]) != 1536 or len(n[1])!=64 :
    print("fehler")
input_data = list(map(lambda x : [x[0][:1024], x[1]], data))
output_data = list(map(lambda x : [x[0][:1024]], data))

model_inputs = tokenizer(input_data[1],truncation=True)

KeyError: ignored